In [ ]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("emmarex/plantdisease")

print("Path to dataset files:", path)

import tensorflow as tf
import numpy as np
import os
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam


# Define image size and batch size
IMAGE_SIZE = (128, 128)
BATCH_SIZE = 32

# Load dataset using ImageDataGenerator
train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_generator = train_datagen.flow_from_directory(
    "C:/Users/suyas/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1",  # Change to your dataset path
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training"
)

val_generator = train_datagen.flow_from_directory(
    "C:/Users/suyas/.cache/kagglehub/datasets/emmarex/plantdisease/versions/1",
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation"
)

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Build CNN Model
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2,2),

    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Conv2D(128, (3,3), activation='relu'),
    MaxPooling2D(2,2),

    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(train_generator.num_classes, activation='softmax')
])

# Compile model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10
)

In [ ]:
import tensorflow as tf
import numpy as np
import os
from google.colab import drive
from datetime import datetime
import matplotlib.pyplot as plt

In [ ]:
# Mount Google Drive
drive.mount('/content/drive')

# Set random seeds for reproducibility
tf.random.set_seed(42)
np.random.seed(42)

# Define the path to your dataset in Google Drive
# Update this path according to your Google Drive structure
base_dir = '/content/drive/MyDrive/Plant'

# Define data directories
train_dir = os.path.join(base_dir, 'Train', 'Train')
test_dir = os.path.join(base_dir, 'Test', 'Test')
valid_dir = os.path.join(base_dir, 'Validation', 'Validation')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Define parameters
img_size = (224, 224)
batch_size = 32
epochs = 1
class_names = ['Healthy', 'Powdery', 'Rust']
num_classes = len(class_names)

# Print directory structure for verification
print("Checking directories...")
print(f"Training directory: {train_dir}")
print(f"Testing directory: {test_dir}")
print(f"Validation directory: {valid_dir}")

# Verify if directories exist
for dir_path in [train_dir, test_dir, valid_dir]:
    if not os.path.exists(dir_path):
        raise ValueError(f"Directory not found: {dir_path}")
    for class_name in class_names:
        class_path = os.path.join(dir_path, class_name)
        if not os.path.exists(class_path):
            raise ValueError(f"Class directory not found: {class_path}")
        num_images = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
        print(f"Found {num_images} images in {class_path}")

# Create data generators
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

print("\nCreating data generators...")

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_names,
    shuffle=True,
    seed=42
)

valid_generator = valid_test_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_names,
    shuffle=False
)

test_generator = valid_test_datagen.flow_from_directory(
    test_dir,
    target_size=img_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=class_names,
    shuffle=False
)

print("\nCreating model...")

# Create the model
model = tf.keras.Sequential([
    # First Convolutional Block
    tf.keras.layers.Conv2D(32, (3, 3), padding='same', input_shape=(*img_size, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    # Second Convolutional Block
    tf.keras.layers.Conv2D(64, (3, 3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    # Third Convolutional Block
    tf.keras.layers.Conv2D(128, (3, 3), padding='same'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Dropout(0.25),

    # Dense Layers
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Activation('relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(num_classes, activation='softmax')
])

# Learning rate scheduler
initial_learning_rate = 0.001
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=1000,
    decay_rate=0.9,
    staircase=True
)

# Compile model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# Create directories for checkpoints and logs in Google Drive
checkpoint_dir = os.path.join(base_dir, 'checkpoints')
logs_dir = os.path.join(base_dir, 'logs')
os.makedirs(checkpoint_dir, exist_ok=True)
os.makedirs(logs_dir, exist_ok=True)

# Callbacks
callbacks = [
    tf.keras.callbacks.ModelCheckpoint(
        os.path.join(checkpoint_dir, 'best_model.h5'),
        save_best_only=True,
        monitor='val_accuracy'
    ),
    tf.keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=10,
        restore_best_weights=True
    ),
    tf.keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.2,
        patience=5,
        min_lr=1e-6
    ),
    tf.keras.callbacks.TensorBoard(
        log_dir=os.path.join(logs_dir, datetime.now().strftime("%Y%m%d-%H%M%S")),
        histogram_freq=1
    )
]

print("\nStarting training...")

# Train the model
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=valid_generator,
    callbacks=callbacks
)

# Evaluate the model
print("\nEvaluating model...")
test_loss, test_accuracy = model.evaluate(test_generator)
print(f"\nTest accuracy: {test_accuracy*100:.2f}%")

# # Save the final model
# final_model_path = os.path.join(base_dir, 'plant_disease_model_final.h5')
# model.save(final_model_path)
# print(f"\nModel saved to {final_model_path}")

# Plot training history
plt.figure(figsize=(12, 4))

# Plot training & validation accuracy
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

# Plot training & validation loss
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')

plt.tight_layout()
plt.show()

print("\nTraining completed!")

Checking directories...
Training directory: /content/drive/MyDrive/Plant/Train/Train
Testing directory: /content/drive/MyDrive/Plant/Test/Test
Validation directory: /content/drive/MyDrive/Plant/Validation/Validation
Found 458 images in /content/drive/MyDrive/Plant/Train/Train/Healthy
Found 430 images in /content/drive/MyDrive/Plant/Train/Train/Powdery
Found 434 images in /content/drive/MyDrive/Plant/Train/Train/Rust
Found 50 images in /content/drive/MyDrive/Plant/Test/Test/Healthy
Found 50 images in /content/drive/MyDrive/Plant/Test/Test/Powdery
Found 50 images in /content/drive/MyDrive/Plant/Test/Test/Rust
Found 20 images in /content/drive/MyDrive/Plant/Validation/Validation/Healthy
Found 20 images in /content/drive/MyDrive/Plant/Validation/Validation/Powdery
Found 20 images in /content/drive/MyDrive/Plant/Validation/Validation/Rust

Creating data generators...
Found 1322 images belonging to 3 classes.
Found 60 images belonging to 3 classes.
Found 150 images belonging to 3 classes.

C